# Manipulating the DB using SqlAlchemy and ORMs

In [1]:
# Fetch configuration from the .env file

import os
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env
host="localhost" # if you are running this script locally but the DB runs on docker then use `localhost` as the hostname. Otherwise use `os.getenv("POSTGRES_HOST")` if the script runs from a docker container started with docker-compose.
database=os.getenv("POSTGRES_DB")
user=os.getenv("POSTGRES_USER")
password=os.getenv("POSTGRES_PASSWORD")
port=os.getenv("POSTGRES_PORT")

In [2]:
# Build the DB url

db_url = 'postgresql+psycopg2://{user}:{password}@{hostname}:{port}/{database_name}'.format(hostname=host, user=user, password=password, database_name=database, port=5432)

In [3]:
# Connect to DB using the DB url
from sqlalchemy import Table, Column, Integer, String, MetaData, create_engine, inspect

engine = create_engine(db_url, echo=False)
inspector = inspect(engine)

In [4]:
# Get all tables from DB
inspector.get_table_names()

['caracteristiques', 'lieux', 'vehicules', 'auth_users']

From the [optional module on SQL](https://train.learn.datascientest.com/notebooks/559/1679)

In [5]:
# Describe a DB table using sqlalchemy ORM

from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column

class Base(DeclarativeBase):
    pass

class User(Base):
    __tablename__ = "auth_users"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str]
    password: Mapped[str]

    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.password!r})"

In [6]:
# Create table based on the ORM model
Base.metadata.create_all(engine)

In [7]:
# Check that table created
inspector.get_table_names()

['caracteristiques', 'lieux', 'vehicules', 'auth_users']

In [8]:
# Add data to the newly created table

from sqlalchemy.orm import Session

with Session(engine) as session:
    user1 = User(
        name="spongebob",
        password="Squarepants",
    )
    user2 = User(
        name="bibi",
        password="blocksberg",
    )

    session.add_all([user1, user2])
    session.commit()

In [9]:
# Fetch data from the DB table

from sqlalchemy import select

stmt = select(User)
for user in session.scalars(stmt):
    print(user)

User(id=1, name='spongebob', fullname='Squarepants')
User(id=2, name='bibi', fullname='blocksberg')
User(id=3, name='spongebob', fullname='Squarepants')
User(id=4, name='bibi', fullname='blocksberg')
User(id=5, name='spongebob', fullname='Squarepants')
User(id=6, name='bibi', fullname='blocksberg')


In [10]:
# Fetch a particular user from the users table

from sqlalchemy import select

stmt = select(User).where(User.name == "bibi")
for user in session.scalars(stmt):
    print(user.password)

blocksberg
blocksberg
blocksberg


In [23]:
# Delete table
# https://docs.sqlalchemy.org/en/20/faq/metadata_schema.html#my-program-is-hanging-when-i-say-table-drop-metadata-drop-all

User.__table__.drop(engine) # this may take ages